# SousChef - End to End Pipeline_4

**Objective**: Our objective for this notebook is to filter the dataframe of 5000+ All Recipes recipes, topic modelled meal types, and predicted cuisines and return a recommended recipe to the user based on the inputs

In [1]:
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer


import pickle

In [26]:
# df = pd.read_csv('datasets/final_recipes.csv')

In [27]:
# df.head()

## EDA

In [28]:
# df.columns

In [29]:
# df.drop(['title_ing_joined'], axis = 1, inplace = True)
# df = df[['title', 'time', 'meal_type', 'cuisine', 'instructions', 'ingredients', 'formatted_text', 'joined']]

In [30]:
# df.columns = ['title', 'time', 'meal_type', 'cuisine','instructions', 'ingredients', 'list_ingredients', 'ingred_string']
# df.head()

In [31]:
# df['id'] = np.random.randint(0,df.shape[0], df.shape[0])

In [32]:
# df.columns

In [33]:
# df = df[['id', 'title', 'time', 'meal_type', 'cuisine', 'instructions', 'ingredients',
#        'list_ingredients', 'ingred_string']]
# df.head()

In [34]:
# df.to_csv('datasets/final_dataset.csv', index = False)

## Cuisine Predictions Based on User Input

In [11]:
# with open('datasets/whats-cooking/train.json', 'r') as f:
#     txt = f.read()
# df_cus = pd.DataFrame(json.loads(txt))
# df_cus.head()

In [12]:
# #cleaning the data
# df_cus['joined'] = df_cus.ingredients.map(lambda x: ' '.join(x))
# df_nb = df_cus[['cuisine','joined']]
# df_nb.head()

In [13]:
# #save new df
# df_nb.to_csv('datasets/final_cuisines.csv', index = False)

In [14]:
# count_vect = CountVectorizer()
# tfidf_transformer = TfidfTransformer()
# X = count_vect.fit_transform(df_nb.joined)
# X = tfidf_transformer.fit_transform(X)
# X.shape

In [15]:
# from sklearn.externals import joblib

# model = joblib.load('cuisine_class.pkl')

In [16]:
# user_input = ['chicken', 'mushroom', 'lettuce']

In [17]:
# user_format = " ".join(user_input)

In [18]:
# user_format

In [19]:
# ch_dict = dict([(y,x) for x,y in enumerate(set(df_nb.cuisine))])

# dec_dict = dict([(x,y) for y,x in ch_dict.items()])

# X_user = np.array(list(user_format))
# X_user = count_vect.transform(X_user)
# X_user = tfidf_transformer.transform(X_user)

In [20]:
# preds = model.predict(X_user)

# y_user = [dec_dict[np.argmax(preds)]]
# cuisine_user = y_user
# cuisine_user

In [21]:
# cuis_str = ''.join(cuisine_user)
# cuis_str

## Recipe Recommender

In [35]:
def cuis_pred(user_input):
    '''function returns the cuisine prediction based on the user input ingredients
    Input: 
        user_input = list of string ingredients
    
    '''
    import pandas as pd
    import numpy as np
    import json
    from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
    from sklearn.externals import joblib
    
    #format the user_input
    user_format = " ".join(user_input)
    
    #call the cuisines file
    df_nb = pd.read_csv('datasets/final_cuisines.csv')
    
    count_vect = CountVectorizer()
    tfidf_transformer = TfidfTransformer()
    
    #transform the text
    X = count_vect.fit_transform(df_nb.joined)
    X = tfidf_transformer.fit_transform(X)
    
    #call model
    model = joblib.load('cuisine_class_att1.pkl')
    
    #transform the user_formats
    ch_dict = dict([(y,x) for x,y in enumerate(set(df_nb.cuisine))])

    dec_dict = dict([(x,y) for y,x in ch_dict.items()])

    X_user = np.array(list(user_format))
    X_user = count_vect.transform(X_user)
    X_user = tfidf_transformer.transform(X_user)
    
    #predictions
    preds = model.predict(X_user)

    y_user = [dec_dict[np.argmax(preds)]]
    
    cuisine = ''.join(y_user)
    
    return cuisine

In [36]:
# Function to filter a dataframe based on a list of ingredients
# to check if there's an ingredient in a recipe of df_in
def filterbyIng(list_ing, cuisine):
    '''function returns another dataframe that includes recipes with, at least, one component from list_ing
    the output dataframe contains at least one ingredient
    Input:
        df_in    : dataframe to examine
        list_ing : list of ingredients,e.g., ['cheese','lettuce','tomato']
    '''
    from itertools import combinations
    
    df_in = pd.read_csv('datasets/final_dataset.csv')
    
    df_out = pd.DataFrame([],columns= df_in.columns)
    #df_out = df_in[df_in['cuisine'] == str(cuisine_user)]
    
    for each in list_ing:
        
        df_containing = df_in[df_in['list_ingredients'].str.contains(each)]
        df_out = df_out.append(df_containing)
        df_out = df_out.drop_duplicates(subset='title')
        
    df_user = df_out[df_out['cuisine']== cuisine]

    return df_user[df_user.time == df_user.time.min()]

In [41]:
user_input = ['chicken', 'mushroom', 'lettuce']
#user_input = ['eggs', 'sugar', 'flour']
#user_input = ['onion','mushroom','tomato']

In [42]:
cuisine = cuis_pred(user_input)

recomm_recipe = filterbyIng(user_input, cuisine)

print('SousChef recommends you make a {} dish based on your listed ingredients! It\'s called {}. \n'.format(recomm_recipe['cuisine'].iloc[0], recomm_recipe['title'].iloc[0]))

print('Don\'t you worry! SousChef has your back, the recipe will only take you {} minutes to make this {}. \n'.format(recomm_recipe['time'].iloc[0], recomm_recipe['meal_type'].iloc[0]))

print('Ingredients: \n')
print(recomm_recipe['ingredients'].iloc[0], '\n')

print('Instructions: \n')
print(recomm_recipe['instructions'].iloc[0], '\n')

SousChef recommends you make a japanese dish based on your listed ingredients! It's called Teriyaki Chicken. 

Don't you worry! SousChef has your back, the recipe will only take you 0 minutes to make this maindish. 

Ingredients: 

['1 (3 pound) whole chicken, cut in half', '¾ cup granulated sugar', '¾ cup soy sauce', '1 tablespoon grated fresh ginger', '2 cloves garlic, minced'] 

Instructions: 

Rinse chicken halves, and pat dry with paper towels. Place chicken cut side down in a 9x13 inch baking dish.
In a medium mixing bowl, combine sugar, soy sauce, grated ginger and garlic. Mix well, and pour mixture over chicken. Cover and refrigerate for at least 3 hours.
Preheat oven to 350 degrees F (175 degrees C).
Bake chicken uncovered in the preheated oven for 1 hour, basting frequently. Test for doneness, making sure there is no pink left in the meat. Let cool slightly, then cut into smaller pieces to serve. 

